# Test different classification algorithms

In [ ]:
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate # train/test split
from sklearn.preprocessing import StandardScaler # scale data
from sklearn.neighbors import KNeighborsClassifier # knn
from sklearn.metrics import classification_report, confusion_matrix # eval performance
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, make_scorer, precision_score, classification_report
from sklearn.svm import SVC # svc
from sklearn import svm

from xgboost import XGBClassifier

In [3]:
# #test new library for benchmarking df
# # from https://nbviewer.jupyter.org/github/JosPolfliet/pandas-profiling/blob/master/examples/meteorites.ipynb
# import pandas_profiling

In [4]:
pd.set_option('display.height', 1000) # make printing better for the entire set of columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
data = pd.DataFrame.from_csv('arabica_to_classify.csv') # check our csv
data.head()


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,country_of_origin,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,cupper_points,total_cup_points,moisture,category_one_defects,category_two_defects,altitude_low_meters,altitude_high_meters,altitude_mean_meters,processing_method_Other,processing_method_Pulped natural / honey,processing_method_Semi-washed / Semi-pulped,processing_method_Washed / Wet,region
0,Ethiopia,8.67,8.83,8.67,8.75,8.50,8.42,10.0,10.0,10.0,8.75,90.58,0.12,0,0,1950.0,2200.0,2075.0,0,0,0,1,africa
1,Ethiopia,8.75,8.67,8.50,8.58,8.42,8.42,10.0,10.0,10.0,8.58,89.92,0.12,0,1,1950.0,2200.0,2075.0,0,0,0,1,africa
3,Ethiopia,8.17,8.58,8.42,8.42,8.50,8.25,10.0,10.0,10.0,8.67,89.00,0.11,0,2,1800.0,2200.0,2000.0,0,0,0,0,africa
4,Ethiopia,8.25,8.50,8.25,8.50,8.42,8.33,10.0,10.0,10.0,8.58,88.83,0.12,0,2,1950.0,2200.0,2075.0,0,0,0,1,africa
9,Ethiopia,8.08,8.58,8.50,8.50,7.67,8.42,10.0,10.0,10.0,8.50,88.25,0.10,0,4,1795.0,1850.0,1822.5,0,0,0,0,africa


In [6]:
# drop cols during refinement here
data_trainer = data.drop(['category_one_defects','category_two_defects','total_cup_points'],axis=1)

In [ ]:
data_trainer.head() #add range(alt)

In [91]:
data_trainer.corr()

,alt_range,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,cupper_points,moisture,altitude_low_meters,altitude_high_meters,altitude_mean_meters,processing_method_Other,processing_method_Pulped natural / honey,processing_method_Semi-washed / Semi-pulped,processing_method_Washed / Wet,region_id
alt_range,1.000000,0.111096,0.073492,0.101751,0.088420,0.099967,0.095558,0.035475,0.017014,0.024381,0.089567,-0.043196,-0.017632,-0.003622,-0.010628,-0.047192,-0.029621,-0.038289,0.085856,0.191528
aroma,0.111096,1.000000,0.730794,0.677853,0.603764,0.551579,0.611399,0.135050,0.197036,0.090089,0.620442,-0.078021,-0.020674,-0.019120,-0.019897,-0.052785,-0.002699,0.032285,-0.051052,0.188929
flavor,0.073492,0.730794,1.000000,0.848174,0.752697,0.685275,0.734379,0.221840,0.301446,0.180586,0.778922,-0.120650,-0.013684,-0.012656,-0.013171,-0.026602,0.008481,0.046016,-0.139939,0.218862
aftertaste,0.101751,0.677853,0.848174,1.000000,0.690328,0.685504,0.764795,0.241180,0.319673,0.191644,0.778498,-0.166180,-0.028069,-0.026647,-0.027359,-0.020345,0.056854,0.043226,-0.120681,0.251334
acidity,0.088420,0.603764,0.752697,0.690328,1.000000,0.617726,0.643872,0.191302,0.167756,0.113706,0.652430,-0.099828,0.002574,0.003814,0.003194,-0.057303,-0.005517,0.008777,-0.021607,0.184458
body,0.099967,0.551579,0.685275,0.685504,0.617726,1.000000,0.702503,0.131102,0.144744,0.086115,0.637715,-0.200925,-0.019658,-0.018260,-0.018959,-0.051480,0.050640,0.035159,-0.135983,0.250960
balance,0.095558,0.611399,0.734379,0.764795,0.643872,0.702503,1.000000,0.241331,0.262809,0.156111,0.719986,-0.216984,-0.017328,-0.015992,-0.016660,-0.020347,0.024910,0.047359,-0.122942,0.268938
uniformity,0.035475,0.135050,0.221840,0.241180,0.191302,0.131102,0.241331,1.000000,0.375281,0.364216,0.206771,-0.005838,-0.012522,-0.012027,-0.012275,-0.039409,0.029326,-0.004501,-0.023793,0.156405
clean_cup,0.017014,0.197036,0.301446,0.319673,0.167756,0.144744,0.262809,0.375281,1.000000,0.429201,0.281215,-0.031121,-0.005381,-0.005143,-0.005262,-0.002999,0.019659,0.030247,-0.057383,0.135357
sweetness,0.024381,0.090089,0.180586,0.191644,0.113706,0.086115,0.156111,0.364216,0.429201,1.000000,0.149870,0.030718,-0.023223,-0.022884,-0.023054,-0.034582,0.016212,0.009952,0.030595,0.004170


In [7]:
# add cols for feature engineering
alt_range = data_trainer['altitude_high_meters']- data_trainer['altitude_low_meters']
data_trainer['alt_range']=alt_range
data_trainer.head()

,country_of_origin,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,cupper_points,moisture,altitude_low_meters,altitude_high_meters,altitude_mean_meters,processing_method_Other,processing_method_Pulped natural / honey,processing_method_Semi-washed / Semi-pulped,processing_method_Washed / Wet,region,alt_range
0,Ethiopia,8.67,8.83,8.67,8.75,8.50,8.42,10.0,10.0,10.0,8.75,0.12,1950.0,2200.0,2075.0,0,0,0,1,africa,250.0
1,Ethiopia,8.75,8.67,8.50,8.58,8.42,8.42,10.0,10.0,10.0,8.58,0.12,1950.0,2200.0,2075.0,0,0,0,1,africa,250.0
3,Ethiopia,8.17,8.58,8.42,8.42,8.50,8.25,10.0,10.0,10.0,8.67,0.11,1800.0,2200.0,2000.0,0,0,0,0,africa,400.0
4,Ethiopia,8.25,8.50,8.25,8.50,8.42,8.33,10.0,10.0,10.0,8.58,0.12,1950.0,2200.0,2075.0,0,0,0,1,africa,250.0
9,Ethiopia,8.08,8.58,8.50,8.50,7.67,8.42,10.0,10.0,10.0,8.50,0.10,1795.0,1850.0,1822.5,0,0,0,0,africa,55.0


In [ ]:
# reorder cols
cols = list(data_trainer.columns.values)
print(cols)

In [8]:
data_trainer = data_trainer[['region','country_of_origin', 'alt_range', 'aroma', 'flavor', 'aftertaste', 'acidity', 'body', 'balance', 'uniformity', 'clean_cup', 'sweetness', 'cupper_points', 'moisture', 'altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters', 'processing_method_Other', 'processing_method_Pulped natural / honey', 'processing_method_Semi-washed / Semi-pulped', 'processing_method_Washed / Wet']]
# reformat order of cols 

In [9]:
#add region column
regions_int = { 'United States': 0, #'north_america',
           'Mexico': 0, #'north_america',
           'China' : 1, #'asia',
           'Taiwan': 1, #'asia',
           'Japan':  1, #'asia',
           'El Salvador':                 2, #'central_america_carib',
           'Nicaragua':                   2, #'central_america_carib',
           'Costa Rica':                  2, #'central_america_carib',
           'Guatemala':                   2, #'central_america_carib',
           'Honduras':                    2, #'central_america_carib',
           'Haiti':                       2, #'central_america_carib',
           'United States (Puerto Rico)': 2, #'central_america_carib',
           'Brazil' : 3, #'south_america',
           'Peru':      3, #'south_america',
           'Colombia':  3, #'south_america',
           'Panama':    3, #'south_america',
           'Ecuador':   3, #'south_america',
           'Ethiopia':  4, #'africa',
           'Kenya':     4, #'africa',
           'Burundi':   4, #'africa',
           'Cote d?Ivoire': 4, #'africa',
           'Rwanda':    4, #'africa',
           'Malawi':    4, #'africa',
           'Uganda':    4, #'africa',
           'Tanzania, United Republic Of': 4, #'africa',
           'Zambia':                       4, #'africa',
           'Thailand':         5, #'southeast_asia',
           'Myanmar':          5, #'southeast_asia',
           'Indonesia':        5, #'southeast_asia',
           'Papua New Guinea': 5, #'southeast_asia',
           'Vietnam':          5, #'southeast_asia',
           'Philippines':      5, #'southeast_asia',
           'Laos':             5, #'southeast_asia'
          }


# create a new column for the region as an ID value for each country
region_id_list=[]
for row in data_trainer['country_of_origin']:
    region_id = regions_int[row]
    region_id_list.append(region_id)
    
data_trainer['region_id'] = region_id_list

In [ ]:
# add flavor v aftertaste
data_trainer['flavor_aftertaste'] = data_trainer['aftertaste']*data_trainer['flavor']
# add flavor v aroma
data_trainer['flavor_aroma'] = data_trainer['flavor']*data_trainer['aroma']
# add flavor v acidity 
data_trainer['flavor_acidity'] = data_trainer['flavor']*data_trainer['acidity']
# add flavor v body
data_trainer['flavor_body'] = data_trainer['flavor']*data_trainer['body']
# flavor v balance
data_trainer['flavor_balance'] = data_trainer['flavor']*data_trainer['balance']
# flavor v cleancup
data_trainer['flavor_clean_cup'] = data_trainer['flavor']*data_trainer['clean_cup']
#cupper v flavor, 
data_trainer['flavor_cupper'] = data_trainer['flavor']*data_trainer['cupper_points']
# add aftertaste w balance
data_trainer['aftertaste_balance'] = data_trainer['aftertaste']*data_trainer['balance']
#cupper v aftertaste
data_trainer['aftertaste_cupper'] = data_trainer['aftertaste']*data_trainer['cupper_points']
# aftertaste vs acidity
data_trainer['aftertaste_acidity'] = data_trainer['aftertaste']*data_trainer['acidity']
# clean cup v aftertaste
data_trainer['aftertaste_clean_cup'] = data_trainer['aftertaste']*data_trainer['clean_cup']
# alt_range * flavor?
data_trainer['aftertaste_alt_range'] = data_trainer['aftertaste']*data_trainer['alt_range']
#cupper points v balance
data_trainer['balance_cupper'] = data_trainer['balance']*data_trainer['cupper_points']
# balance and body
data_trainer['balance_body'] = data_trainer['balance']*data_trainer['body']

#need to add items as interaction terms to feed as cols to the clf below

data_trainer.head()

In [10]:
X_cols = data_trainer[['aroma', 'flavor', 'aftertaste', 'acidity', 'body', 'balance', 'uniformity', \
          'clean_cup', 'sweetness', 'cupper_points', 'moisture', 'altitude_low_meters', 'altitude_high_meters', \
          'altitude_mean_meters', 'processing_method_Other', 'processing_method_Pulped natural / honey', \
          'processing_method_Semi-washed / Semi-pulped', 'processing_method_Washed / Wet', \
          'alt_range']]

# to add: 'flavor_balance', 'flavor_body', 'flavor_acidity', 'flavor_aroma',  'flavor_aftertaste'
#         'balance_body', 'balance_cupper', 'aftertaste_alt_range','aftertaste_clean_cup', 'aftertaste_acidity',
#       'aftertaste_cupper', 'aftertaste_balance', 'flavor_cupper', 'flavor_clean_cup','balance_body' 

In [11]:
# X contains the attributes, y contains the labels
# [row_start:row_end, col_start:col_end]

y = data_trainer['region_id'].values # region_id
X = X_cols.values # features

#data_trainer.corr()

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) # train, test split 

In [87]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

parameters = {'C':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]} # svc params
# A parameter grid for XGBoost
params = {
        'max_depth': [3, 4, 5]
        }
xgb = XGBClassifier(objective='multi:softmax', num_class=6) #learning_rate=0.02, n_estimators=600, objective='multi:softmax', num_class=6

#svc = svm.SVC() 
gbc = GradientBoostingClassifier()#loss='deviance', learning_rate=0.1, n_estimators=5

#clf = GridSearchCV(svc, parameters, cv=5)
clf = GridSearchCV(gbc, params, cv=5)

clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=xgb, param_grid=params, return_train_score=True)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, num_class=6, objective='multi:softmax',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 4, 5]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [88]:
#parameters = {'C':[1, 20]} # need help understanding this param object

clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(C=1.0), param_grid=parameters, return_train_score=True)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [89]:
y_pred=clf.predict(X_test) # precision score

In [90]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.81      0.68      0.74        62
          1       0.73      0.62      0.67        13
          2       0.69      0.77      0.73        48
          3       0.79      0.70      0.74        53
          4       0.52      0.79      0.62        19
          5       0.38      0.50      0.43         6

avg / total       0.73      0.71      0.71       201



In [ ]:
#clf.best_params_
#clf.best_estimator_

## try diff classifiers individually

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) # train, test split 

In [66]:
# scaler = StandardScaler()  # normalize
# scaler.fit(X_train)

# X_train = scaler.transform(X_train)  
# X_test = scaler.transform(X_test)

### decision tree

In [67]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test) 

In [68]:
# evaluate performance  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

[[24  1 10  1  7  3]
 [ 3  6  1  5  0  4]
 [13  2 36  3  4  2]
 [ 4  0  4 29  0  3]
 [ 4  1  7  3  8  0]
 [ 5  0  3  3  0  2]]
             precision    recall  f1-score   support

          0       0.45      0.52      0.48        46
          1       0.60      0.32      0.41        19
          2       0.59      0.60      0.60        60
          3       0.66      0.72      0.69        40
          4       0.42      0.35      0.38        23
          5       0.14      0.15      0.15        13

avg / total       0.53      0.52      0.52       201



### SVC

In [69]:
classifier = SVC(C=3)
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test) 

In [70]:
# evaluate performance  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

[[26  1 12  2  4  1]
 [ 9  3  1  4  0  2]
 [15  0 42  2  1  0]
 [10  0  0 27  2  1]
 [ 8  1  3  2  9  0]
 [ 8  0  2  1  0  2]]
             precision    recall  f1-score   support

          0       0.34      0.57      0.43        46
          1       0.60      0.16      0.25        19
          2       0.70      0.70      0.70        60
          3       0.71      0.68      0.69        40
          4       0.56      0.39      0.46        23
          5       0.33      0.15      0.21        13

avg / total       0.57      0.54      0.53       201



### K nearest neighbors classification

In [71]:
classifier = KNeighborsClassifier(n_neighbors=4)
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test) 

In [72]:
# evaluate performance  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

[[26  1  9  3  5  2]
 [ 6  5  2  4  0  2]
 [13  0 39  4  2  2]
 [ 7  2  1 27  1  2]
 [ 8  0  4  6  5  0]
 [ 5  0  3  3  0  2]]
             precision    recall  f1-score   support

          0       0.40      0.57      0.47        46
          1       0.62      0.26      0.37        19
          2       0.67      0.65      0.66        60
          3       0.57      0.68      0.62        40
          4       0.38      0.22      0.28        23
          5       0.20      0.15      0.17        13

avg / total       0.52      0.52      0.51       201



### Random Forest Classifier

In [73]:
classifier = RandomForestClassifier(n_estimators=5)
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test) 

In [74]:
# evaluate performance  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

[[21  1 15  3  6  0]
 [ 3  6  4  6  0  0]
 [12  3 39  2  3  1]
 [ 3  4  2 30  1  0]
 [ 4  0  5  5  9  0]
 [ 7  0  2  3  1  0]]
             precision    recall  f1-score   support

          0       0.42      0.46      0.44        46
          1       0.43      0.32      0.36        19
          2       0.58      0.65      0.61        60
          3       0.61      0.75      0.67        40
          4       0.45      0.39      0.42        23
          5       0.00      0.00      0.00        13

avg / total       0.48      0.52      0.50       201



### XGBoost

In [75]:
classifier = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=5)
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test) 

In [76]:
# evaluate performance  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred)) 

[[27  0  9  3  6  1]
 [ 4  4  2  7  2  0]
 [15  0 41  1  3  0]
 [ 4  0  3 32  1  0]
 [ 5  0  4  3 10  1]
 [ 6  0  1  5  0  1]]
             precision    recall  f1-score   support

          0       0.44      0.59      0.50        46
          1       1.00      0.21      0.35        19
          2       0.68      0.68      0.68        60
          3       0.63      0.80      0.70        40
          4       0.45      0.43      0.44        23
          5       0.33      0.08      0.12        13

avg / total       0.60      0.57      0.55       201



### Graph for best k number - irrelevant?

In [ ]:
# error = []

# # Calculating error for K values between 1 and 40

# for i in range(1, 40):  
#     knn = KNeighborsClassifier(n_neighbors=i)
#     knn.fit(X_train, y_train)
#     pred_i = knn.predict(X_test)
#     error.append(np.mean(pred_i != y_test))

In [ ]:
# plt.figure(figsize=(17, 7))  
# plt.plot(range(1, 40), error, color='black', linestyle='dashed', marker='.', markerfacecolor='green', markersize=7)
# plt.title('K Value Error Change')
# plt.xlabel('K Value')
# plt.ylabel('Mean Error')

In [ ]:
# Ugh, time waster

# # cross validate, compare models and changes across models if desired
# # this cell does everything the next heading and below do, but better
# from sklearn.model_selection import cross_val_score
# from sklearn import metrics
# import random
# from sklearn.ensemble import GradientBoostingClassifier

# def score_func(y_true, y_pred):
#     return np.mean(metrics.precision_score(y_true, y_pred, average=None))

# def variance_metric(y_true, y_pred):
#     return np.sqrt(np.var(y_true - y_pred))

# scoring = {'precision': make_scorer(score_func),
#            'std_dev': make_scorer(variance_metric)
#           }

# ######
# # run the code below for several different random seeds
# ######

# seed_list = np.arange(1,25,1)
# scores = []

# for seed in seed_list:
#     random.seed(seed)
#     svc = SVC(C=5)
# #     knn = KNeighborsClassifier(n_neighbors=2)
# #     rf = RandomForestClassifier(n_estimators=5)
# #     dt = DecisionTreeClassifier()
# add xgboost
# # gridsearchCV is better
    
#     scores.append(cross_validate(svc, X_train, y_train, scoring=scoring, cv=5)) 
# #     scores.append(cross_validate(knn, X_train, y_train, scoring=scoring, cv=5)) #svc clearly outperforms
# #     scores.append(cross_validate(rf, X_train, y_train, scoring=scoring, cv=5))
# #     scores.append(cross_validate(dt, X_train, y_train, scoring=scoring, cv=5))
    
# print('test_precision_avg')
# for score_set in scores:
#     print(np.mean(score_set['test_precision']))
    
# print('std_dev')
# for score_set in scores:
#     print(score_set['test_std_dev'])